In [11]:
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark import SparkFiles
import pandas as pd
import numpy as np
import requests
from IPython.display import HTML
from transformers import LongformerForMaskedLM, LongformerTokenizer, AutoModelForMaskedLM, BertTokenizer, AutoTokenizer
import seaborn as sn
import matplotlib.pyplot as plt
from datasets import Dataset, load_dataset

In [2]:
import os
import sys

os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [7]:
ds = load_dataset("Czunzun/Reuters_first_512")
ds = ds['train'].train_test_split(test_size=.05)

In [8]:
ds

DatasetDict({
    train: Dataset({
        features: ['first_512', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 92592
    })
    test: Dataset({
        features: ['first_512', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 4874
    })
})

In [12]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling

#Importing the models
lf_model = LongformerForMaskedLM.from_pretrained("allenai/longformer-base-4096")
bert_model = AutoModelForMaskedLM.from_pretrained("google-bert/bert-base-uncased")

#Importing BERT tokenizer
bert_tokenizer = BertTokenizer.from_pretrained('google-bert/bert-base-uncased')

#Importing Data Collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=bert_tokenizer,
    mlm=True,
    mlm_probability=.15
)


#defining training arguments
training_args = TrainingArguments("test-trainer", push_to_hub=True, hub_model_id="finbert2_v1")
#defining trainer
trainer = Trainer(
    model=bert_model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=ds['train'],
    eval_dataset=ds['test']

)
trainer.train()


Some weights of the model checkpoint at google-bert/bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Step,Training Loss


KeyboardInterrupt: 